In [2]:
library(destiny)
library(slingshot)
library(stringr)
library(Seurat)
library(SingleCellExperiment)
library(ggplot2)
library(readr)
library(gam)
library(zellkonverter)

Registered S3 method overwritten by 'zellkonverter':
  method                from      
  py_to_r.numpy.ndarray reticulate



In [3]:
hammond_dir = 'data/other_data/Hammond/'
hammond_zip_files = list.files(hammond_dir)

# unzip files if needed
# lapply(hammond_zip_files, function(f) {
#     if (str_ends(f, 'gz')) {
#         system(paste0('gzip -d ', hammond_dir, f))
#     }
# })

hammond_files = list.files(hammond_dir)

# The following code is modified based on 
# https://github.com/michael-r-odea/Green_ODea_2022/blob/main/Hammond.R
# Publication link: https://www.nature.com/articles/s41593-022-01091-9

In [16]:
preprocess_data = function(list_of_data, label='', k = 10000) {
    sample_ids = names(list_of_data)

    hammond_data_s = lapply(
        list_of_data, CreateSeuratObject, project = "Hammond")

    # add timepoint and sample_id metadata to each seurat object in list
    for (i in seq_along(hammond_data_s)){
      hammond_data_s[[i]]@meta.data[["label"]] = label
      hammond_data_s[[i]]@meta.data[["sample_id"]] = sample_ids[i]
    }

    # Calculate the percentage of UMIs mapping to mitochondrial genes here and 
    # store it in percent.mt  
    for (i in seq_along(hammond_data_s)){
      hammond_data_s[[i]]@meta.data[["percent.mt"]] = PercentageFeatureSet(
          object = hammond_data_s[[i]], pattern = "^mt.")
    }

    # @Hao 01/04/2023: Reversing the order of celling filtering and merge 
    # because it hited a bug in the original order

    # based on QC plots, filtering out cells with <400 genes or > 3000 unique 
    # genes; cells with > 10,000 UMIs, and cells with greater than 3% of reads 
    # mapping to mitochondrial genes 
    hammond_data_s = lapply(hammond_data_s, function(x) {
        subset(
            x, 
            subset = nFeature_RNA > 400 & nFeature_RNA < 3000 & 
            nCount_RNA < 10000 & percent.mt < 3)
    })

    # find highly variable genes
    features = SelectIntegrationFeatures(
        object.list = hammond_data_s, nfeatures = k
    )
    hammond = merge(x = hammond_data_s[[1]], 
                    y = hammond_data_s[2:(length(list_of_data))])

    # Log-normalize data
    hammond = NormalizeData(hammond)

    # Convert seurat back to scexperiment
    hammond_sc = as.SingleCellExperiment(hammond)
    
    return(list(hammond_sc, features))
}

save_data = function(processed_dt, label) {
    normalized = as.data.frame(logcounts(processed_dt[[1]]))
    save_dir = 'data/other_data/Hammond_processed/final/'
    write.csv(normalized, paste0(save_dir, 'male_', label, '_data.csv'))
}

In [6]:
hammond_data = lapply(hammond_files, function(x) {
    file_meta = str_split(x, '_')[[1]]
    if (tolower(file_meta[3]) %in% c('male', 'm')) {
        read.table(
            paste0(hammond_dir, x), header = TRUE, row.names = 1, sep = "\t")
    }
})

# get sample names from input file names
names(hammond_data) = hammond_files %>%
  str_replace(pattern = ".dge.txt", replacement = "") %>%
  str_replace(pattern = "GSM......._", replacement = "")
hammond_data = hammond_data[lengths(hammond_data) > 0]


In [17]:
save_data(preprocess_data(hammond_data[1:4], 'male_e14'), 'male_e14')
save_data(preprocess_data(hammond_data[5:8], 'male_p45'), 'male_p45')
save_data(preprocess_data(hammond_data[9:12], 'male_p30'), 'male_p30')
save_data(preprocess_data(hammond_data[13:16], 'male_p100'), 'male_p100')
save_data(preprocess_data(hammond_data[17:20], 'male_old'), 'male_old')

No variable features found for object1 in the object.list. Running FindVariableFeatures ...

No variable features found for object2 in the object.list. Running FindVariableFeatures ...

No variable features found for object3 in the object.list. Running FindVariableFeatures ...

No variable features found for object4 in the object.list. Running FindVariableFeatures ...

Warning message in CheckDuplicateCellNames(object.list = objects):
“Some cell names are duplicated across objects provided. Renaming to enforce unique cell names.”
Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 1.5 GiB”
No variable features found for object1 in the object.list. Running FindVariableFeatures ...

No variable features found for object2 in the object.list. Running FindVariableFeatures ...

No variable features found for object3 in the object.list. Running FindVariableFeatures ...

No variable features found for object4 in the object.list. Running FindVariableFeatures 